In [1]:
import numpy as np
import pandas as pd

In [4]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric, add_changepoints_to_plot, plot_plotly
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## Read in the data from Kaggle

In [12]:
df = pd.read_csv('2019_nCoV_data.csv')
conf_df = pd.read_csv('time_series_2019_ncov_confirmed.csv')
deaths_df = pd.read_csv('time_series_2019_ncov_deaths.csv')
recv_df = pd.read_csv('time_series_2019_ncov_recovered.csv')

In [26]:
df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020 12:00:00,Anhui,China,01/22/2020 12:00:00,1.0,0.0,0.0
1,2,01/22/2020 12:00:00,Beijing,China,01/22/2020 12:00:00,14.0,0.0,0.0
2,3,01/22/2020 12:00:00,Chongqing,China,01/22/2020 12:00:00,6.0,0.0,0.0
3,4,01/22/2020 12:00:00,Fujian,China,01/22/2020 12:00:00,1.0,0.0,0.0
4,5,01/22/2020 12:00:00,Gansu,China,01/22/2020 12:00:00,0.0,0.0,0.0


In [41]:
dates = list(conf_df.columns[4:])
dates1 = list(recv_df.columns[4:])

### We would want to aggregate the confirmed/death/recovered data under one date frame

In [43]:
conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates1, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates1, var_name='Date', value_name='Recovered')

In [47]:
conf_df

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Boston, MA",US,42.36010,-71.0589,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
68,"San Benito, CA",US,36.57610,-120.9876,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
69,NaN,Belgium,50.50390,4.4699,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
70,"Madison, WI",US,43.07310,-89.4012,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1,1,1,1,1,1


In [44]:
conf_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,Anhui,Mainland China,31.82571,117.2264,1/21/20 22:00,NaN
1,Beijing,Mainland China,40.18238,116.4142,1/21/20 22:00,10.0
2,Chongqing,Mainland China,30.05718,107.8740,1/21/20 22:00,5.0
3,Fujian,Mainland China,26.07783,117.9895,1/21/20 22:00,NaN
4,Gansu,Mainland China,36.06110,103.8343,1/21/20 22:00,NaN
...,...,...,...,...,...,...
2587,"Boston, MA",US,42.36010,-71.0589,02/09/20 23:20,1.0
2588,"San Benito, CA",US,36.57610,-120.9876,02/09/20 23:20,2.0
2589,NaN,Belgium,50.50390,4.4699,02/09/20 23:20,1.0
2590,"Madison, WI",US,43.07310,-89.4012,02/09/20 23:20,1.0


In [50]:
full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], axis=1, sort=False)

In [52]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.82571,117.2264,1/21/20 22:00,NaN,NaN,NaN
1,Beijing,Mainland China,40.18238,116.4142,1/21/20 22:00,10.0,NaN,NaN
2,Chongqing,Mainland China,30.05718,107.8740,1/21/20 22:00,5.0,NaN,NaN
3,Fujian,Mainland China,26.07783,117.9895,1/21/20 22:00,NaN,NaN,NaN
4,Gansu,Mainland China,36.06110,103.8343,1/21/20 22:00,NaN,NaN,NaN


In [53]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [55]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.82571,117.2264,2020-01-21 22:00:00,NaN,NaN,NaN
1,Beijing,Mainland China,40.18238,116.4142,2020-01-21 22:00:00,10.0,NaN,NaN
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-21 22:00:00,5.0,NaN,NaN
3,Fujian,Mainland China,26.07783,117.9895,2020-01-21 22:00:00,NaN,NaN,NaN
4,Gansu,Mainland China,36.06110,103.8343,2020-01-21 22:00:00,NaN,NaN,NaN


In [56]:
full_table.dtypes

Province/State            object
Country/Region            object
Lat                      float64
Long                     float64
Date              datetime64[ns]
Confirmed                float64
Deaths                   float64
Recovered                float64
dtype: object

In [59]:
full_table['Country/Region'].unique()

array(['Mainland China', 'Thailand', 'Japan', 'South Korea', 'Taiwan',
       'US', 'Macau', 'Hong Kong', 'Singapore', 'Vietnam', 'France',
       'Nepal', 'Malaysia', 'Canada', 'Australia', 'Cambodia',
       'Sri Lanka', 'Germany', 'Finland', 'United Arab Emirates',
       'Philippines', 'India', 'Italy', 'UK', 'Russia', 'Sweden', 'Spain',
       'Belgium', 'Others'], dtype=object)

## Add a column for population of the city

In [60]:
full_table['Province/State'].unique()

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin',
       'Liaoning', 'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong',
       'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Xinjiang',
       'Yunnan', 'Zhejiang', nan, 'Taiwan', 'Seattle, WA', 'Chicago, IL',
       'Tempe, AZ', 'Macau', 'Hong Kong', 'Toronto, ON',
       'British Columbia', 'Orange, CA', 'Los Angeles, CA',
       'New South Wales', 'Victoria', 'Queensland', 'London, ON',
       'Santa Clara, CA', 'South Australia', 'Boston, MA',
       'San Benito, CA', 'Madison, WI', 'Diamond Princess cruise ship'],
      dtype=object)

In [66]:
def label_race (row):
    if row['Province/State'] == 'Anhui' :
        return 62.0
    if row['Province/State'] == 'Beijing' :
        return 21.54
    if row['Province/State'] == 'Chongqing':
        return 30.48
    if row['Province/State'] == 'Fujian':
        return 38.56
    if row['Province/State']  == 'Gansu':
        return 25.58
    if row['Province/State']  == 'Guangdong':
        return 113.46 
    if row['Province/State']  == 'Guangxi':
        return 48.38 
    if row['Province/State']  == 'Guizhou':
        return 34.75 
    if row['Province/State']  == 'Hainan':
        return 9.3
    if row['Province/State']  == 'Hebei':
        return 74.7
    if row['Province/State']  == 'Heilongjiang':
        return 38.31
    if row['Province/State']  == 'Henan':
        return 94 
    if row['Province/State']  == '':
        return 113.46 
    if row['Province/State']  == '':
        return 113.46 
    if row['Province/State']  == '':
        return 113.46 
    return 10

In [67]:
full_table.apply (lambda row: label_race(row), axis=1)

0       62.00
1       21.54
2       30.48
3       38.56
4       25.58
        ...  
2587    10.00
2588    10.00
2589    10.00
2590    10.00
2591    10.00
Length: 2592, dtype: float64

In [ ]:
# filling missing values with 0 in columns ('Confirmed', 'Deaths', 'Recovered')
full_table[['Confirmed', 'Deaths', 'Recovered']] = full_table[['Confirmed', 'Deaths', 'Recovered']].fillna(0)

# adding two more columns
full_table['Deaths to Confirmed'] = round(full_table['Deaths']/full_table['Confirmed'], 3)
full_table['Recovered to Confirmed'] = round(full_table['Deaths']/full_table['Confirmed'], 3)